# Import

In [3]:
import json

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder

item_cat = pd.read_csv("/Users/leo/samurai/kaggle/pfs/data/item_categories.csv")
items = pd.read_csv("/Users/leo/samurai/kaggle/pfs/data/items.csv")
train = pd.read_csv("/Users/leo/samurai/kaggle/pfs/data/sales_train.csv")
shops = pd.read_csv("/Users/leo/samurai/kaggle/pfs/data/shops.csv")
test = pd.read_csv("/Users/leo/samurai/kaggle/pfs/data/test.csv")
sample_sub = pd.read_csv("/Users/leo/samurai/kaggle/pfs/data/sample_submission.csv")

# Submission

In [4]:
sample_sub.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


# EDA

## item_cat

In [5]:
item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [6]:
item_cat.shape

(84, 2)

In [7]:
item_cat.item_category_name.nunique()

84

In [8]:
item_cat.item_category_name.unique()

array(['PC - Гарнитуры/Наушники', 'Аксессуары - PS2', 'Аксессуары - PS3',
       'Аксессуары - PS4', 'Аксессуары - PSP', 'Аксессуары - PSVita',
       'Аксессуары - XBOX 360', 'Аксессуары - XBOX ONE', 'Билеты (Цифра)',
       'Доставка товара', 'Игровые консоли - PS2',
       'Игровые консоли - PS3', 'Игровые консоли - PS4',
       'Игровые консоли - PSP', 'Игровые консоли - PSVita',
       'Игровые консоли - XBOX 360', 'Игровые консоли - XBOX ONE',
       'Игровые консоли - Прочие', 'Игры - PS2', 'Игры - PS3',
       'Игры - PS4', 'Игры - PSP', 'Игры - PSVita', 'Игры - XBOX 360',
       'Игры - XBOX ONE', 'Игры - Аксессуары для игр',
       'Игры Android - Цифра', 'Игры MAC - Цифра',
       'Игры PC - Дополнительные издания',
       'Игры PC - Коллекционные издания', 'Игры PC - Стандартные издания',
       'Игры PC - Цифра', 'Карты оплаты (Кино, Музыка, Игры)',
       'Карты оплаты - Live!', 'Карты оплаты - Live! (Цифра)',
       'Карты оплаты - PSN', 'Карты оплаты - Windows (Цифра)',

### adding big_category to item_cat

In [9]:
big_category = [x.split("-")[0].strip() for x in item_cat.item_category_name]

In [10]:
big_category

['PC',
 'Аксессуары',
 'Аксессуары',
 'Аксессуары',
 'Аксессуары',
 'Аксессуары',
 'Аксессуары',
 'Аксессуары',
 'Билеты (Цифра)',
 'Доставка товара',
 'Игровые консоли',
 'Игровые консоли',
 'Игровые консоли',
 'Игровые консоли',
 'Игровые консоли',
 'Игровые консоли',
 'Игровые консоли',
 'Игровые консоли',
 'Игры',
 'Игры',
 'Игры',
 'Игры',
 'Игры',
 'Игры',
 'Игры',
 'Игры',
 'Игры Android',
 'Игры MAC',
 'Игры PC',
 'Игры PC',
 'Игры PC',
 'Игры PC',
 'Карты оплаты (Кино, Музыка, Игры)',
 'Карты оплаты',
 'Карты оплаты',
 'Карты оплаты',
 'Карты оплаты',
 'Кино',
 'Кино',
 'Кино',
 'Кино',
 'Кино',
 'Книги',
 'Книги',
 'Книги',
 'Книги',
 'Книги',
 'Книги',
 'Книги',
 'Книги',
 'Книги',
 'Книги',
 'Книги',
 'Книги',
 'Книги',
 'Музыка',
 'Музыка',
 'Музыка',
 'Музыка',
 'Музыка',
 'Музыка',
 'Подарки',
 'Подарки',
 'Подарки',
 'Подарки',
 'Подарки',
 'Подарки',
 'Подарки',
 'Подарки',
 'Подарки',
 'Подарки',
 'Подарки',
 'Подарки',
 'Программы',
 'Программы',
 'Программы',
 'Прог

In [11]:
item_cat["big_category"] = big_category

In [12]:
item_cat.head()

,item_category_name,item_category_id,big_category
0,PC - Гарнитуры/Наушники,0,PC
1,Аксессуары - PS2,1,Аксессуары
2,Аксессуары - PS3,2,Аксессуары
3,Аксессуары - PS4,3,Аксессуары
4,Аксессуары - PSP,4,Аксессуары


In [18]:
item_cat.big_category.nunique()

20

In [19]:
with open("/Users/leo/samurai/pfs/data/input/big_category_rus_eng.json") as json_file:
    trans = json.load(json_file)

In [20]:
item_cat.big_category.replace(to_replace=trans, inplace=True)

In [21]:
item_cat.head()

,item_category_name,item_category_id,big_category
0,PC - Гарнитуры/Наушники,0,pc
1,Аксессуары - PS2,1,accesories
2,Аксессуары - PS3,2,accesories
3,Аксессуары - PS4,3,accesories
4,Аксессуары - PSP,4,accesories


In [22]:
item_cat.big_category.unique()

array(['pc', 'accesories', 'tickets', 'delivery', 'game consoles',
       'games', 'android_games', 'mac_games', 'pc_games', 'payment_cards',
       'gift_cards', 'cinema', 'books', 'music', 'present', 'programs',
       'services', 'cd', 'dvd', 'battery'], dtype=object)

### adding sub_category to item_cat

In [45]:
item_cat.head(30)

,item_category_name,item_category_id,big_category
0,PC - Гарнитуры/Наушники,0,pc
1,Аксессуары - PS2,1,accesories
2,Аксессуары - PS3,2,accesories
3,Аксессуары - PS4,3,accesories
4,Аксессуары - PSP,4,accesories
5,Аксессуары - PSVita,5,accesories
6,Аксессуары - XBOX 360,6,accesories
7,Аксессуары - XBOX ONE,7,accesories
8,Билеты (Цифра),8,tickets
9,Доставка товара,9,delivery


In [39]:
item_cat.item_category_name.unique()

array(['PC - Гарнитуры/Наушники', 'Аксессуары - PS2', 'Аксессуары - PS3',
       'Аксессуары - PS4', 'Аксессуары - PSP', 'Аксессуары - PSVita',
       'Аксессуары - XBOX 360', 'Аксессуары - XBOX ONE', 'Билеты (Цифра)',
       'Доставка товара', 'Игровые консоли - PS2',
       'Игровые консоли - PS3', 'Игровые консоли - PS4',
       'Игровые консоли - PSP', 'Игровые консоли - PSVita',
       'Игровые консоли - XBOX 360', 'Игровые консоли - XBOX ONE',
       'Игровые консоли - Прочие', 'Игры - PS2', 'Игры - PS3',
       'Игры - PS4', 'Игры - PSP', 'Игры - PSVita', 'Игры - XBOX 360',
       'Игры - XBOX ONE', 'Игры - Аксессуары для игр',
       'Игры Android - Цифра', 'Игры MAC - Цифра',
       'Игры PC - Дополнительные издания',
       'Игры PC - Коллекционные издания', 'Игры PC - Стандартные издания',
       'Игры PC - Цифра', 'Карты оплаты (Кино, Музыка, Игры)',
       'Карты оплаты - Live!', 'Карты оплаты - Live! (Цифра)',
       'Карты оплаты - PSN', 'Карты оплаты - Windows (Цифра)',

In [36]:
sub_cat = [
    x.split("-")[1].strip() if len(x.split("-")) > 1 else x.split("-")[0].strip()
    for x in item_cat.item_category_name
]

In [41]:
sub_cat

['Гарнитуры/Наушники',
 'PS2',
 'PS3',
 'PS4',
 'PSP',
 'PSVita',
 'XBOX 360',
 'XBOX ONE',
 'Билеты (Цифра)',
 'Доставка товара',
 'PS2',
 'PS3',
 'PS4',
 'PSP',
 'PSVita',
 'XBOX 360',
 'XBOX ONE',
 'Прочие',
 'PS2',
 'PS3',
 'PS4',
 'PSP',
 'PSVita',
 'XBOX 360',
 'XBOX ONE',
 'Аксессуары для игр',
 'Цифра',
 'Цифра',
 'Дополнительные издания',
 'Коллекционные издания',
 'Стандартные издания',
 'Цифра',
 'Карты оплаты (Кино, Музыка, Игры)',
 'Live!',
 'Live! (Цифра)',
 'PSN',
 'Windows (Цифра)',
 'Blu',
 'Blu',
 'Blu',
 'DVD',
 'Коллекционное',
 'Артбуки, энциклопедии',
 'Аудиокниги',
 'Аудиокниги (Цифра)',
 'Аудиокниги 1С',
 'Бизнес литература',
 'Комиксы, манга',
 'Компьютерная литература',
 'Методические материалы 1С',
 'Открытки',
 'Познавательная литература',
 'Путеводители',
 'Художественная литература',
 'Цифра',
 'CD локального производства',
 'CD фирменного производства',
 'MP3',
 'Винил',
 'Музыкальное видео',
 'Подарочные издания',
 'Атрибутика',
 'Гаджеты, роботы, спорт'

In [46]:
item_cat['sub_category'] = sub_cat

In [47]:
item_cat.head()

,item_category_name,item_category_id,big_category,sub_category
0,PC - Гарнитуры/Наушники,0,pc,Гарнитуры/Наушники
1,Аксессуары - PS2,1,accesories,PS2
2,Аксессуары - PS3,2,accesories,PS3
3,Аксессуары - PS4,3,accesories,PS4
4,Аксессуары - PSP,4,accesories,PSP


## items

In [62]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [63]:
items.shape

(22170, 3)

### merging item_cat and items to make full_items

In [64]:
full_items = pd.merge(item_cat, items, on="item_category_id", how="left")

In [65]:
full_items.head()

,item_category_name,item_category_id,big_category,item_name,item_id
0,PC - Гарнитуры/Наушники,0,pc,PC: Гарнитура HyperX Cloud Core игровая стерео...,5441
1,PC - Гарнитуры/Наушники,0,pc,Наушники PHILIPS SBC HC8680,16255
2,PC - Гарнитуры/Наушники,0,pc,Наушники RITMIX RH-120,16256
3,PC - Гарнитуры/Наушники,0,pc,Наушники RITMIX RH-124 Black,16257
4,Аксессуары - PS2,1,accesories,PS2: Карта памяти 8 Мб черная (Memory Card 8Mb...,5606


In [66]:
# dropping item category name and item name for now

In [67]:
full_items.drop(columns=["item_category_name", "item_name"], inplace=True)

In [68]:
full_items.head()

,item_category_id,big_category,item_id
0,0,pc,5441
1,0,pc,16255
2,0,pc,16256
3,0,pc,16257
4,1,accesories,5606


## shops

In [69]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [70]:
shops.shape

(60, 2)

### adding city to shops

In [71]:
city = [x.split(" ")[0].strip() for x in shops.shop_name]

In [72]:
shops["city"] = city

In [73]:
shops.head()

,shop_name,shop_id,city
0,"!Якутск Орджоникидзе, 56 фран",0,!Якутск
1,"!Якутск ТЦ ""Центральный"" фран",1,!Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский


In [74]:
shops.city.nunique()

32

In [75]:
shops.city.value_counts()

Москва              13
Тюмень               3
Воронеж              3
РостовНаДону         3
Казань               2
Жуковский            2
Уфа                  2
Н.Новгород           2
Красноярск           2
Новосибирск          2
СПб                  2
!Якутск              2
Якутск               2
Самара               2
Сергиев              1
Волжский             1
Мытищи               1
Коломна              1
Выездная             1
Сургут               1
Интернет-магазин     1
Цифровой             1
Химки                1
Калуга               1
Ярославль            1
Балашиха             1
Адыгея               1
Чехов                1
Томск                1
Вологда              1
Омск                 1
Курск                1
Name: city, dtype: int64

In [76]:
# removing typos

In [77]:
shops.loc[shops.city == "!Якутск", "city"] = "Якутск"

In [78]:
shops.city.value_counts()

Москва              13
Якутск               4
Воронеж              3
РостовНаДону         3
Тюмень               3
СПб                  2
Красноярск           2
Жуковский            2
Н.Новгород           2
Новосибирск          2
Казань               2
Самара               2
Уфа                  2
Сергиев              1
Выездная             1
Сургут               1
Коломна              1
Мытищи               1
Химки                1
Цифровой             1
Курск                1
Калуга               1
Ярославль            1
Балашиха             1
Адыгея               1
Интернет-магазин     1
Омск                 1
Вологда              1
Томск                1
Волжский             1
Чехов                1
Name: city, dtype: int64

In [79]:
shops.drop(columns="shop_name", inplace=True)

In [80]:
shops.head()

,shop_id,city
0,0,Якутск
1,1,Якутск
2,2,Адыгея
3,3,Балашиха
4,4,Волжский


## train

In [81]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [82]:
train.shape

(2935849, 6)

### Merging full_items and shops

In [83]:
shops.head()

,shop_id,city
0,0,Якутск
1,1,Якутск
2,2,Адыгея
3,3,Балашиха
4,4,Волжский


In [84]:
full_items.head()

,item_category_id,big_category,item_id
0,0,pc,5441
1,0,pc,16255
2,0,pc,16256
3,0,pc,16257
4,1,accesories,5606


In [85]:
full_train = pd.merge(train, full_items, on="item_id", how="left")

In [86]:
full_train = pd.merge(full_train, shops, on="shop_id", how="left")

In [87]:
full_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,big_category,city
0,02.01.2013,0,59,22154,999.00,1.0,37,cinema,Ярославль
1,03.01.2013,0,25,2552,899.00,1.0,58,music,Москва
2,05.01.2013,0,25,2552,899.00,-1.0,58,music,Москва
3,06.01.2013,0,25,2554,1709.05,1.0,58,music,Москва
4,15.01.2013,0,25,2555,1099.00,1.0,56,music,Москва


In [88]:
full_train.city.describe(include=object)

count     2935849
unique         31
top        Москва
freq       996636
Name: city, dtype: object

In [89]:
full_train.big_category.describe(include=object)

count     2935849
unique         20
top        cinema
freq       833416
Name: big_category, dtype: object

### adding column item_cnt per date_block_num

In [90]:
data = pd.pivot_table(
    data=train,
    values="item_cnt_day",
    index=["shop_id", "item_id"],
    columns="date_block_num",
    aggfunc="sum",
).fillna(0)

In [91]:
data.reset_index(inplace=True)

In [92]:
data.head()

date_block_num,shop_id,item_id,0,1,2,3,4,5,6,7,...,24,25,26,27,28,29,30,31,32,33
0,0,30,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,31,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,32,6.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,33,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,35,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
full_train = pd.merge(full_train, data, on=["shop_id", "item_id"], how="left")

In [94]:
full_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,big_category,city,0,...,24,25,26,27,28,29,30,31,32,33
0,02.01.2013,0,59,22154,999.00,1.0,37,cinema,Ярославль,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,03.01.2013,0,25,2552,899.00,1.0,58,music,Москва,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,05.01.2013,0,25,2552,899.00,-1.0,58,music,Москва,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,06.01.2013,0,25,2554,1709.05,1.0,58,music,Москва,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15.01.2013,0,25,2555,1099.00,1.0,56,music,Москва,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
full_train.columns = [
    "date",
    "date_block_num",
    "shop_id",
    "item_id",
    "item_price",
    "item_cnt_day",
    "item_category_id",
    "big_category",
    "city",
] + ["item_cnt_month_" + str(x) for x in range(0, 34)]

In [96]:
full_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,big_category,city,item_cnt_month_0,...,item_cnt_month_24,item_cnt_month_25,item_cnt_month_26,item_cnt_month_27,item_cnt_month_28,item_cnt_month_29,item_cnt_month_30,item_cnt_month_31,item_cnt_month_32,item_cnt_month_33
0,02.01.2013,0,59,22154,999.00,1.0,37,cinema,Ярославль,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,03.01.2013,0,25,2552,899.00,1.0,58,music,Москва,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,05.01.2013,0,25,2552,899.00,-1.0,58,music,Москва,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,06.01.2013,0,25,2554,1709.05,1.0,58,music,Москва,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15.01.2013,0,25,2555,1099.00,1.0,56,music,Москва,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Encoding Features

In [97]:
full_train["big_category"] = LabelEncoder().fit_transform(full_train.big_category)

In [98]:
full_train["city"] = LabelEncoder().fit_transform(full_train.city)

In [99]:
full_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,big_category,city,item_cnt_month_0,...,item_cnt_month_24,item_cnt_month_25,item_cnt_month_26,item_cnt_month_27,item_cnt_month_28,item_cnt_month_29,item_cnt_month_30,item_cnt_month_31,item_cnt_month_32,item_cnt_month_33
0,02.01.2013,0,59,22154,999.00,1.0,37,5,30,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,03.01.2013,0,25,2552,899.00,1.0,58,12,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,05.01.2013,0,25,2552,899.00,-1.0,58,12,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,06.01.2013,0,25,2554,1709.05,1.0,58,12,13,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15.01.2013,0,25,2555,1099.00,1.0,56,12,13,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
full_train.city.describe().astype(int)

count    2935849
mean          16
std            7
min            0
25%           13
50%           13
75%           23
max           30
Name: city, dtype: int64

In [101]:
full_train.big_category.describe().astype(int)

count    2935849
mean          10
std            4
min            0
25%            5
50%            9
75%           15
max           19
Name: big_category, dtype: int64

# Creating train files

In [102]:
full_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,big_category,city,item_cnt_month_0,...,item_cnt_month_24,item_cnt_month_25,item_cnt_month_26,item_cnt_month_27,item_cnt_month_28,item_cnt_month_29,item_cnt_month_30,item_cnt_month_31,item_cnt_month_32,item_cnt_month_33
0,02.01.2013,0,59,22154,999.00,1.0,37,5,30,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,03.01.2013,0,25,2552,899.00,1.0,58,12,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,05.01.2013,0,25,2552,899.00,-1.0,58,12,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,06.01.2013,0,25,2554,1709.05,1.0,58,12,13,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15.01.2013,0,25,2555,1099.00,1.0,56,12,13,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
# Dropping shop_id and item_id duplicates so that the same product doens't exist twice

In [104]:
full_train = full_train.drop_duplicates(subset=["shop_id", "item_id"])

In [105]:
full_train.reset_index(drop=True, inplace=True)

In [106]:
full_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,big_category,city,item_cnt_month_0,...,item_cnt_month_24,item_cnt_month_25,item_cnt_month_26,item_cnt_month_27,item_cnt_month_28,item_cnt_month_29,item_cnt_month_30,item_cnt_month_31,item_cnt_month_32,item_cnt_month_33
0,02.01.2013,0,59,22154,999.00,1.0,37,5,30,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,03.01.2013,0,25,2552,899.00,1.0,58,12,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,06.01.2013,0,25,2554,1709.05,1.0,58,12,13,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15.01.2013,0,25,2555,1099.00,1.0,56,12,13,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.01.2013,0,25,2564,349.00,1.0,59,12,13,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,01.10.2015,33,25,7640,4040.00,1.0,64,16,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
424120,24.10.2015,33,25,7231,999.00,1.0,3,0,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
424121,18.10.2015,33,25,7327,349.00,1.0,55,12,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
424122,24.10.2015,33,25,7315,399.00,1.0,55,12,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [107]:
# clipping item_cnt from (0,20) since thats what Kaggle does when reviewing

In [108]:
full_train.iloc[:, 9:] = full_train.iloc[:, 9:].clip(0, 20).copy()

In [109]:
full_train.item_cnt_month_25.describe()

count    424124.000000
mean          0.186599
std           0.896127
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          20.000000
Name: item_cnt_month_25, dtype: float64

In [110]:
# dropping 'date','date_block_num','shop_id','item_id','item_cnt_day','item_price'

In [111]:
X_train = full_train.iloc[:, :-1].drop(
    columns=[
        "date",
        "date_block_num",
        "shop_id",
        "item_id",
        "item_cnt_day",
        "item_price",
    ]
)

In [112]:
print(X_train.shape)
X_train.head()

(424124, 36)


,item_category_id,big_category,city,item_cnt_month_0,item_cnt_month_1,item_cnt_month_2,item_cnt_month_3,item_cnt_month_4,item_cnt_month_5,item_cnt_month_6,...,item_cnt_month_23,item_cnt_month_24,item_cnt_month_25,item_cnt_month_26,item_cnt_month_27,item_cnt_month_28,item_cnt_month_29,item_cnt_month_30,item_cnt_month_31,item_cnt_month_32
0,37,5,30,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,58,12,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,58,12,13,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,56,12,13,1.0,1.0,1.0,4.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,59,12,13,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
y_train = full_train.iloc[:, -1].drop(
    columns=[
        "date",
        "date_block_num",
        "shop_id",
        "item_id",
        "item_cnt_day",
        "item_price",
    ]
)

In [114]:
print(y_train.shape)
y_train.head()

(424124,)


0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: item_cnt_month_33, dtype: float64

In [115]:
X_test = full_train.drop(
    columns=[
        "date",
        "date_block_num",
        "shop_id",
        "item_id",
        "item_cnt_day",
        "item_price",
        "item_cnt_month_0",
    ]
)

In [116]:
print(X_test.shape)
X_test.head()

(424124, 36)


,item_category_id,big_category,city,item_cnt_month_1,item_cnt_month_2,item_cnt_month_3,item_cnt_month_4,item_cnt_month_5,item_cnt_month_6,item_cnt_month_7,...,item_cnt_month_24,item_cnt_month_25,item_cnt_month_26,item_cnt_month_27,item_cnt_month_28,item_cnt_month_29,item_cnt_month_30,item_cnt_month_31,item_cnt_month_32,item_cnt_month_33
0,37,5,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,58,12,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,58,12,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,56,12,13,1.0,1.0,4.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,59,12,13,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Fitting lightbgm

In [117]:
LGBM = lgb.LGBMRegressor()

In [118]:
cv = cross_val_score(LGBM, X_train, y_train)
print(cv)
print(cv.mean())

[ 0.62345468  0.62149673 -1.50841986 -0.49661568  0.07188708]
-0.1376394112483727


In [119]:
LGBM.fit(X_train, y_train)

LGBMRegressor()

# Submission


In [120]:
y_test = LGBM.predict(X_test)

In [121]:
y_test = pd.DataFrame(y_test)

In [122]:
y_test.rename(columns={0: "item_cnt_month"}, inplace=True)

In [123]:
y_test

,item_cnt_month
0,0.235822
1,0.267072
2,0.267072
3,0.003778
4,0.039460
...,...
424119,0.365762
424120,0.376890
424121,0.863950
424122,0.340535


In [124]:
test_final = pd.concat([full_train, y_test], axis=1)

In [125]:
test_final

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,big_category,city,item_cnt_month_0,...,item_cnt_month_25,item_cnt_month_26,item_cnt_month_27,item_cnt_month_28,item_cnt_month_29,item_cnt_month_30,item_cnt_month_31,item_cnt_month_32,item_cnt_month_33,item_cnt_month
0,02.01.2013,0,59,22154,999.00,1.0,37,5,30,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.235822
1,03.01.2013,0,25,2552,899.00,1.0,58,12,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.267072
2,06.01.2013,0,25,2554,1709.05,1.0,58,12,13,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.267072
3,15.01.2013,0,25,2555,1099.00,1.0,56,12,13,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003778
4,10.01.2013,0,25,2564,349.00,1.0,59,12,13,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,01.10.2015,33,25,7640,4040.00,1.0,64,16,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.365762
424120,24.10.2015,33,25,7231,999.00,1.0,3,0,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.376890
424121,18.10.2015,33,25,7327,349.00,1.0,55,12,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.863950
424122,24.10.2015,33,25,7315,399.00,1.0,55,12,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.340535


In [126]:
submission = pd.merge(
    test,
    test_final[["shop_id", "item_id", "item_cnt_month"]],
    on=["shop_id", "item_id"],
    how="left",
)

In [127]:
submission.fillna(0, inplace=True)

In [128]:
submission.head()

,ID,shop_id,item_id,item_cnt_month
0,0,5,5037,0.362819
1,1,5,5320,0.000000
2,2,5,5233,0.701316
3,3,5,5232,0.170781
4,4,5,5268,0.000000


In [129]:
submission.drop(columns=["item_id", "shop_id"], inplace=True)

In [130]:
submission.head()

,ID,item_cnt_month
0,0,0.362819
1,1,0.000000
2,2,0.701316
3,3,0.170781
4,4,0.000000


In [131]:
submission.to_csv("submission.csv", index=False)